TrieConstruction(Patterns)
    Trie ← a graph consisting of a single node root
    for each string Pattern in Patterns
        currentNode ← root
        for i ← 0 to |Pattern| - 1
            currentSymbol ← Pattern[i]
            if there is an outgoing edge from currentNode with label currentSymbol
                currentNode ← ending node of this edge
            else
                add a new node newNode to Trie
                add a new edge from currentNode to newNode with label currentSymbol
                currentNode ← newNode
    return Trie

In [1]:
from typing import List, Dict, Iterable, Tuple, Set

In [ ]:
##DEBUG TODO you need reduce memory allocation - store edges using indices and length instead

In [55]:
def Trie_create_edge(Trie: dict,parent:int,child:int,nucleotide:str):
    if parent in Trie:
        Trie[parent].append([nucleotide,child])
    else:
        Trie[parent] = [[nucleotide,child]]
    return Trie,child

def Trie_return_next_node(Trie:dict,currentNode:int,nucleotide:str):
    if currentNode in Trie:
        for edge in Trie[currentNode]:
            if edge[0]==nucleotide:
                return edge[1]
 #returns original currentNode if edge doesn't exist
    return -1

def dict_to_edge_list(Trie:dict):
    edge_list = []
    for key,value in Trie.items():
        for edge in value:
            edge_list.append((key,edge[1],edge[0]))
    return edge_list


In [71]:
def TrieConstruction_dict(Patterns: List[str]):
    Trie = {0:[]}
    num_nodes = 0
    is_leaf = {0:True}
    for pattern in Patterns:
        currentNode = 0
        for i in range(0,len(pattern)):
            currentSymbol = pattern[i]
            nextNode = Trie_return_next_node(Trie,currentNode,currentSymbol)
            if nextNode!=-1:
                currentNode=nextNode
            else:
                is_leaf[currentNode] = False
                Trie,currentNode = Trie_create_edge(Trie,currentNode,num_nodes+1,currentSymbol)
                is_leaf[currentNode] = True
                num_nodes+=1
    return Trie,is_leaf

In [74]:
def TrieConstruction(Patterns: List[str]) -> List[Tuple[int, int, str]]:
    return dict_to_edge_list(TrieConstruction_dict(Patterns)[0])

In [90]:
TrieConstruction(['ATAGA','ATC','GAT'])


[(0, 1, 'A'),
 (0, 7, 'G'),
 (1, 2, 'T'),
 (2, 3, 'A'),
 (2, 6, 'C'),
 (3, 4, 'G'),
 (4, 5, 'A'),
 (7, 8, 'A'),
 (8, 9, 'T')]

PrefixTrieMatching(Text, Trie)
    symbol ← first letter of Text
    v ← root of Trie
    while forever
        if v is a leaf in Trie
            output the pattern spelled by the path from the root to v
        else if there is an edge (v, w) in Trie labeled by symbol
            symbol ← next letter of Text
            v ← w
        else
            return "no matches found"

In [137]:
def Trie_leaf(Trie:List[Tuple[int, int, str]]):
    output_leaf = [False for _ in range(max([edge[1] for edge in Trie])+1)]
    for edge in Trie:
        start,end,nucleotide = edge
        output_leaf[start] = False
        output_leaf[end] = True
    return output_leaf

def does_edge_exist(Trie:List[Tuple[int, int, str]],v:int,symbol:str)->int:
    for edge in Trie:
        start,end,nucleotide = edge
        if start==v and nucleotide==symbol:
            return end
    return -1


In [162]:
def prefix_trie_matching(text:str,Trie:List[Tuple[int, int, str]]): #draw out tree and figure bug out
    leaves = Trie_leaf(Trie)
    ind = 0
    symbol = text[ind]
    v = 0
    match_output = ''
    while True:
        w = does_edge_exist(Trie,v,symbol)
        if w!=-1:
            match_output = match_output + symbol
            v = w
            if leaves[v]:
                return match_output
            if ind<len(text)-1:
                ind+=1
                symbol = text[ind] 
        else:
            return "no match found"


In [163]:
prefix_trie_matching('T',[(0, 1, 'T'), (1, 2, 'A')])

'no match found'

In [139]:
Trie = [(0, 1, 'A'),
 (0, 7, 'G'),
 (1, 2, 'T'),
 (2, 3, 'A'),
 (2, 6, 'C'),
 (3, 4, 'G'),
 (4, 5, 'A'),
 (7, 8, 'A'),
 (8, 9, 'T')]
prefix_trie_matching('ATAGC',Trie)

'no match found'

TrieMatching(Text, Trie)
    while Text is nonempty
        PrefixTrieMatching(Text, Trie)
        remove first symbol from Text

In [159]:
def trie_matching(text: str, patterns: List[str]) -> Dict[str, List[int]]:
    Trie = TrieConstruction(patterns)
    print(Trie)
    output_dict = {pattern:[] for pattern in patterns}
    i = 0
    while text:
        print(text)
        matched_pattern = prefix_trie_matching(text,Trie)
        if matched_pattern!= "no match found":
            output_dict[matched_pattern].append(i)
        i+=1
        text = text[1:]
    return output_dict

## Suffix Trie 
using OOP 

In [418]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.value = None
    def return_value(self):
        return self.value
    def make_leaf(self,i:int):
        if not self.children and self.value==None:
            self.value = i
    def add_edge(self,symbol:str,child,pos:int):
        self.children[symbol]=[pos,child]
    def return_edge(self,symbol:str):
        if symbol not in self.children:
            return -1
        pos,childNode = self.children[symbol]
        return childNode
    def iter_outgoing_edges(self):
        #print(self.children)
        return self.children
    def merge(self,new_symbol,new_child):
        pos = self.children.pop(new_symbol[0])[0]
        self.children[new_symbol] = [pos,new_child]
        #del self.children[new_symbol[0]]
    def find_edge(self,child):
        for symbol,edge in self.children.items():
            if edge[1]==child:
                return symbol
    def return_child_nodes(self): #return 1st degree children
        child_nodes = []
        for edge in self.children.values():
            child_nodes.append(edge[1])
        return child_nodes
class Modified_Suffix_Trie:
    def __init__(self):
        self.root = TrieNode()
        self.nodes = [self.root]
    def return_root(self):
        return self.root
    def iter_nodes(self):
        return self.nodes
    #@staticmethod
    def insert_edge(self,parent:TrieNode,symbol:str,pos:int):
        child = TrieNode()
        parent.add_edge(symbol,child,pos)
        self.nodes.append(child)
        return child
    #@staticmethod
    def merge_edges(self,path:List[Tuple[TrieNode,TrieNode]]): #DEBUG TODO implement this using edges not nodes
        maximal_edge_word = ''
        for edge in path:
            v,w = edge
            maximal_edge_word+=v.find_edge(w)
            self.nodes.remove(w)
            del w
        #print(maximal_edge_word)
        start = path[0][0]
        end = path[-1][1]
        start.merge(maximal_edge_word,end)
    def is_node_1_v_1(self,node:TrieNode):
            #checks if node is internal              #checks if node exactly one outgoing edge
        if (node!=self.root or node.value!=None) and len(node.iter_outgoing_edges())==1:
            return True
        return False
    def print_trie(self, node=None, indent=""):
        if node is None:
            node = self.root

        for symbol, (pos, child) in node.children.items():
            print(f"{indent}{symbol} (Pos: {pos}) --> Node Value: {child.value if child.value is not None else 'None'}")
            self.print_trie(child, indent + "  ")
    
    def edge_list(self, node=None, symbols=None):
        if node is None:
            node = self.root
        if symbols is None:
            symbols = []
        for symbol, (_, child) in node.children.items():
            symbols.append(symbol)
            self.edge_list(child, symbols)
        return symbols
    """
    def unique_edge_dict(self, node=None, edge_dict=None):
        if node is None:
            node = self.root
        if edge_dict is None:
            edge_dict = {}
        
        for symbol, (pos, child) in node.children.items():
            # Directly proceed without altering the symbol, even if it ends with '$'
            if symbol in edge_dict:
                # Ensure the position list exists and append if not already present
                if pos not in edge_dict[symbol]:
                    edge_dict[symbol].append(pos)
            else:
                edge_dict[symbol] = [pos]
            
            # Recursively process child nodes
            self.unique_edge_dict(child, edge_dict)

        return edge_dict
    """
    def print_node_list(self,path:List[TrieNode]):
        for node in path:
            print(node.iter_outgoing_edges())
            if node.value()!=None:
                print(node.value())

ModifiedSuffixTrieConstruction(Text)
    Trie ← a graph consisting of a single node root
    for i ← 0 to |Text| - 1
        currentNode ← root
        for j ← i to |Text| - 1
            currentSymbol ← j-th symbol of Text
            if there is an outgoing edge from currentNode labeled by currentSymbol
                currentNode ← ending node of this edge
            else
                add a new node newNode to Trie
                add an edge newEdge connecting currentNode to newNode in Trie
                Symbol(newEdge) ← currentSymbol
                Position(newEdge) ← j
                currentNode ← newNode
        if currentNode is a leaf in Trie
            assign label i to this leaf
    return Trie

In [455]:
def modified_suffix_trie_construction(text:str):
    Trie = Modified_Suffix_Trie()
    for i in range(len(text)): #suffix ind
        currentNode = Trie.return_root() 
        for j in range(i,len(text)):#suffix ind till end of string
            currentSymbol=text[j]
            nextNode = currentNode.return_edge(currentSymbol)
            if nextNode!=-1:
                currentNode = nextNode
            else:
                currentNode = Trie.insert_edge(currentNode,currentSymbol,j)
        currentNode.make_leaf(i)
    return Trie

In [456]:
def modified_prefix_trie_construction(text:str):
    Trie = Modified_Suffix_Trie()
    for i in range(len(text)): #suffix ind
        print(i,text[i])
        currentNode = Trie.return_root() 
        for j in range(0,i+1):#suffix ind till end of string
            currentSymbol=text[j]
            print(currentSymbol)
            nextNode = currentNode.return_edge(currentSymbol)
            if nextNode!=-1:
                currentNode = nextNode
            else:
                currentNode = Trie.insert_edge(currentNode,currentSymbol,j)
        currentNode.make_leaf(i)
    return Trie

In [454]:
papa_trie = modified_suffix_trie_construction('atp$')
papa_trie.print_trie()
len(papa_trie.iter_nodes())

0 a
a
1 t
a
t
2 p
a
t
p
3 $
a
t
p
$
a (Pos: 0) --> Node Value: 0
  t (Pos: 1) --> Node Value: 1
    p (Pos: 2) --> Node Value: 2
      $ (Pos: 3) --> Node Value: 3


5

Modified suffix TREE <= consolidating non-branching paths in modified suffix TRIE into one edge 

1-in-1-out node = node's indegree and out degree is 1 (->o->) \
in a tree internal nodes are 1-in-1-out, root and leaves are not

MaximalNonBranchingPaths(Graph)
    Paths ← empty list
    for each node v in Graph
        if v is not a 1-in-1-out node
            if out(v) > 0
                for each outgoing edge (v, w) from v
                    NonBranchingPath ← the path consisting of single edge (v, w)
                    while w is a 1-in-1-out node
                        extend NonBranchingPath by the edge (w, u) 
                        w ← u
                    add NonBranchingPath to the set Paths
    for each isolated cycle Cycle in Graph
        add Cycle to Paths
    return Paths

In [364]:
def maximal_non_branching_paths(Trie: Modified_Suffix_Trie):
    paths = []
    count = 0
    for v in Trie.iter_nodes():
        if not Trie.is_node_1_v_1(v):
            if v.iter_outgoing_edges():
                non_branching_path=[]
                for symbol,edge in v.iter_outgoing_edges().items():
                    pos,w = edge
                    non_branching_path.append((v,w))
                    while Trie.is_node_1_v_1(w):
                        u = [edge[1] for edge in w.iter_outgoing_edges().values()][0] #returns only child node of w
                        #print("u",u.iter_outgoing_edges())
                        non_branching_path.append((w,u))
                        w = u
                    #print('oop')
                    #print('append path len',len(non_branching_path))
                    if len(non_branching_path)>0:
                        paths.append(non_branching_path) 
                        non_branching_path=[]
                        count+=1

    return paths

ModifiedSuffixTreeConstruction(Text)
    Trie ← ModifiedSuffixTrieConstruction
    for each non-branching path Path in Trie
        substitute Path by a single edge e connecting the first and last nodes of Path
        Position(e) ← Position(first edge of Path)
        Length(e) ← number of edges of Path
    return Trie

In [390]:
def modified_suffix_tree_construction(text:str):
    Trie = modified_suffix_trie_construction(text)
    for path in maximal_non_branching_paths(Trie):
        Trie.merge_edges(path=path)
    return Trie 

In [389]:
Trie = modified_suffix_tree_construction("panamabananas$")
print(Trie.edge_list())

count 17
----------------maximal_non_branching_paths------------------
0
panamabananas$
1
a
2
na
3
mabananas$
4
bananas$
5
s$
6
$
7
na
8
mabananas$
9
bananas$
10
s$
11
mabananas$
12
nas$
13
s$
14
mabananas$
15
nas$
16
s$
['panamabananas$', 'a', 'na', 'mabananas$', 'nas$', 's$', 'mabananas$', 'bananas$', 's$', 'na', 'mabananas$', 'nas$', 's$', 'mabananas$', 'bananas$', 's$', '$']


Longest repeat problem
Trie <- build suffix tree from Text


In [422]:
def find_longest_repeat(node, path="", longest_repeat={"string": "", "length": 0}):
    # Base case: Leaf node
    if len(node.children) == 0:
        return
    # Internal node with more than one child
    if len(node.children) > 1:
        current_length = len(path)
        if current_length > longest_repeat["length"]:
            longest_repeat["string"] = path
            longest_repeat["length"] = current_length

    # Recursive case: Traverse children
    for symbol, (pos, child) in node.children.items():
        find_longest_repeat(child, path + symbol, longest_repeat)

    return longest_repeat["string"]

In [429]:
def longest_repeat(text: str) -> str:
    Trie =  modified_suffix_tree_construction(text+"$")
    Trie.print_trie()
    return find_longest_repeat(Trie.return_root())

#### longest commong substring

In [459]:
def longest_shared_substring(text1: str, text2: str) -> str:
    # Concatenate texts with a unique delimiter
    combined_text = text1 + "#" + text2 + "$"
    trie = modified_suffix_tree_construction(combined_text)

    # Initialize the result variables
    longest_shared = {"length": 0, "substring": ""}

    def dfs(node, path=""):
        # Base case: If a leaf node, return a set indicating the origin of the leaf
        if len(node.children) == 0:
            return set(["1" if node.value < len(text1) else "2"])
        
        origins = set()
        for symbol, (_, child) in node.children.items():
            child_origins = dfs(child, path + symbol)
            origins |= child_origins

            # If this node has descendants from both texts, check if it's the longest
            if "1" in child_origins and "2" in child_origins and len(path + symbol) > longest_shared["length"]:
                longest_shared["length"] = len(path + symbol)
                longest_shared["substring"] = path + symbol

        return origins

    # Start DFS from the root
    dfs(trie.return_root())

    return longest_shared["substring"]

In [460]:
longest_shared_substring('GAGA','CTCT')

count 14
----------------maximal_non_branching_paths------------------


''

In [461]:
longest_shared_substring('TCGGTAGATTGCGCCCACTC','AGGGGCTCGCAGTGTAAGAA')

count 64
----------------maximal_non_branching_paths------------------


'TCG'

### Sorted Suffix Array

In [212]:
def suffix_array(text: str) -> List[int]:
    sarr = [i for i in range(len(text))]
    sarr.sort(key=lambda item: text[item:])
    return sarr

In [213]:
suffix_array('AACGATAGCGGTAGA$')

[15, 14, 0, 1, 12, 6, 4, 2, 8, 13, 3, 7, 9, 10, 11, 5]

In [439]:
def prefix_array(text: str) -> List[int]:
    parr = [i for i in range(len(text))]
    parr.sort(key=lambda item: text[:item])
    return parr

In [441]:
prefix_array('cat$')

[0, 1, 2, 3]

### BWT 
genome compression of repeats 

In [225]:
def burrows_wheeler_transform(text: str) -> str:
    sarr = [i for i in range(len(text))]
    sarr.sort(key=lambda item: text[item:]+text[:item])
    #for i in sarr:
        #print(text[i:]+text[:i],text[i-1])
    bwtarr = ''.join([text[i-1] for i in sarr])
    return bwtarr

In [226]:
burrows_wheeler_transform('panamabananas$')

$panamabananas s
abananas$panam m
amabananas$pan n
anamabananas$p p
ananas$panamab b
anas$panamaban n
as$panamabanan n
bananas$panama a
mabananas$pana a
namabananas$pa a
nanas$panamaba a
nas$panamabana a
panamabananas$ $
s$panamabanana a


'smnpbnnaaaaa$a'

In [462]:
from typing import List

def bwt_from_suffixarray(text: str, suffix_array: List[int]) -> str:
    text += '#'
    bwt_result = ''
    for i in suffix_array:
        if i == 0:
            bwt_result += '#'
        else:
            bwt_result += text[i - 1]
    
    return bwt_result


In [463]:
bwt_from_suffixarray("bioinf",[6,0,5,3,1,4,2])

'f#nobii'